# Hi :)

----

# Import libraries

In [1]:
import numpy as np
import pandas as pd

----

These steps till next seperating line are for Kalman Filter implementation(run if you don't have KalmanFilter-SisFall.csv file)

----

----

# Loading the dataset

In [2]:
# # Loading the dataset
# SisFall_Dataset= pd.read_csv('Dataset/SisFall_dataset.csv')
# # Showing 5 random rows
# SisFall_Dataset.sample(5, random_state= 48)

In [3]:
# # Making a copy of original dataset to work with
# df= SisFall_Dataset.copy()

In [4]:
# # Checking the number of rows and columns
# df.shape

# Kalman Filter

Kalman Filter Implementation on three axis

Initialization

In [5]:
# # Creating a function to implement Kalman Filter
# def run_kalman_filter_vector(measurements, R_value, A, H, Q, x0, P0):
#     x= x0.copy()
#     P= P0.copy()
#     estimates= []

#     for z in measurements:
#         # Predict
#         x= A @ x
#         P= A @ P @ A.T + Q

#         # Update
#         y= np.array(z).reshape(-1, 1) - H @ x
#         S= H @ P @ H.T + R_value
#         K= P @ H.T @ np.linalg.inv(S)
#         x= x + K @ y
#         P= (np.eye(P.shape[0]) - K @ H) @ P

#         estimates.append(x.flatten())  # Store estimated state

#     return estimates

In [6]:
# A= np.eye(3)   # No dynamics for now
# Q= np.eye(3) * 0.001
# x0= np.zeros((3, 1))
# P0= np.eye(3)

# # Same H matrix as above
# H= np.array([
#     [1, 0, 0], [0, 1, 0], [0, 0, 1],  # accel1
#     [1, 0, 0], [0, 1, 0], [0, 0, 1],  # accel2
#     [1, 0, 0], [0, 1, 0], [0, 0, 1],  # gyro
# ])

# R= np.diag([0.05] * 6 + [0.1] * 3)  # More trust in accelerometers

In [7]:
# # Getting the measurements a specific order for Kalman Filter implementation
# measurements= df[['ADXL345_x', 'ADXL345_y', 'ADXL345_z', 'MMA8451Q_x', 'MMA8451Q_y', 'MMA8451Q_z', 'ITG3200_x', 'ITG3200_y', 'ITG3200_z']].values

Kalman Filter Main loop

In [8]:
# estimates= run_kalman_filter_vector(measurements, R, A, H, Q, x0, P0)

Adding Kalman Filter data to dataset

In [9]:
# # Adding Kalman Filter data
# df[['x_kalman', 'y_kalman', 'z_kalman']]=np.array(estimates).round(2)
# # Fixing the dataset columns
# df= df[[col for col in df.columns if col != 'Situation'] + ['Situation']]

In [10]:
# df.head()

In [11]:
# # Saving the dataset
# df.to_csv('Dataset/KalmanFilter-SisFall.csv', index= False)

----

----

If you have KalmanFilter-SisFall.csv file, you can work form here

----

# Loading the dataset

In [2]:
# Loading the dataset
KalmanFilter_SisFall= pd.read_csv('Dataset/KalmanFilter-SisFall.csv')
# Showing 5 random row of dataset
KalmanFilter_SisFall.sample(5, random_state= 48)

,ADXL345_x,ADXL345_y,ADXL345_z,ITG3200_x,ITG3200_y,ITG3200_z,MMA8451Q_x,MMA8451Q_y,MMA8451Q_z,x_kalman,y_kalman,z_kalman,Situation
636970,3,-237,-149,-120,-122,-96,-78,-881,-437,-52.55,-473.54,-247.41,Fall
734642,-22,59,-283,-43,18,-4,-181,259,-979,-89.62,131.23,-504.66,Fall
1069635,37,-348,-103,702,-766,437,133,-1329,-305,74.21,-828.16,-158.67,Not Fall
793919,279,41,-3,-28,46,-10,982,172,153,495.69,94.04,55.20,Fall
541529,7,-429,-144,-295,960,240,-64,-1627,-429,31.64,-598.62,-94.81,Fall


In [24]:
# Making a copy of original dataset to work with
df= KalmanFilter_SisFall.copy()

# Feature Calculation

ITG data

In [37]:
window_size= 200
overlap= 50
stride= window_size - overlap

features= []

# Get signals and labels
a_x= df['ITG3200_x'].values
a_y= df['ITG3200_y'].values
a_z= df['ITG3200_z'].values
situation= df['Situation'].values

num_samples= len(df)
num_windows= (num_samples - window_size) // stride + 1  # excludes partial windows

for i in range(num_windows):
    start= i * stride
    end= start + window_size

    # Check if full window fits
    if end > num_samples:
        break

    # Extract window data
    window_ax= a_x[start : end]
    window_ay= a_y[start : end]
    window_az= a_z[start : end]
    window_situation= situation[start : end]

    # --- Condition 1: Ensure situation is consistent in window ---
    if len(set(window_situation)) > 1:
        continue  # skip this window

    # --- F1: Sum vector magnitude (RMS) ---
    magnitudes= np.sqrt(window_ax**2 + window_ay**2 + window_az**2)
    F1= np.mean(magnitudes).round(2)

    # --- F2: Orientation angle ---
    angles= np.arctan2(np.sqrt(window_ax**2 + window_az**2), -window_ay)
    F2= np.mean(angles).round(2)

    # --- F3: Std deviation magnitude ---
    std_x= np.std(window_ax)
    std_y= np.std(window_ay)
    std_z= np.std(window_az)
    F3= np.sqrt(std_x**2 + std_y**2 + std_z**2).round(2)

    # Get window label (same for all rows in valid window)
    label= window_situation[0]

    # Store feature and label
    features.append([F1, F2, F3, label, start])

# Convert to DataFrame
ITG_features_df= pd.DataFrame(features, columns= [
    'F1_sum_vector_magnitude',
    'F2_orientation_angle',
    'F3_std_magnitude',
    'Situation',
    'window_start_index'
])

ITG_features_df.sample(5, random_state= 48)

,F1_sum_vector_magnitude,F2_orientation_angle,F3_std_magnitude,Situation,window_start_index
1151,500.51,1.62,671.18,Fall,172650
3842,752.52,1.79,770.46,Fall,576300
5832,63.63,1.74,26.27,Fall,874800
10870,1112.90,1.51,1174.01,Not Fall,1630650
5683,372.60,1.63,339.63,Fall,852450


Kalman Filter data

In [36]:
window_size= 200
overlap= 50
stride= window_size - overlap

features= []

# Get signals and labels
a_x= df['x_kalman'].values
a_y= df['y_kalman'].values
a_z= df['z_kalman'].values
situation= df['Situation'].values

num_samples= len(df)
num_windows= (num_samples - window_size) // stride + 1  # excludes partial windows

for i in range(num_windows):
    start= i * stride
    end= start + window_size

    # Check if full window fits
    if end > num_samples:
        break

    # Extract window data
    window_ax= a_x[start : end]
    window_ay= a_y[start : end]
    window_az= a_z[start : end]
    window_situation= situation[start : end]

    # --- Condition 1: Ensure situation is consistent in window ---
    if len(set(window_situation)) > 1:
        continue  # skip this window

    # --- F1: Sum vector magnitude (RMS) ---
    magnitudes= np.sqrt(window_ax**2 + window_ay**2 + window_az**2)
    F1= np.mean(magnitudes).round(2)

    # --- F2: Orientation angle ---
    angles= np.arctan2(np.sqrt(window_ax**2 + window_az**2), -window_ay)
    F2= np.mean(angles).round(2)

    # --- F3: Std deviation magnitude ---
    std_x= np.std(window_ax)
    std_y= np.std(window_ay)
    std_z= np.std(window_az)
    F3= np.sqrt(std_x**2 + std_y**2 + std_z**2).round(2)

    # Get window label (same for all rows in valid window)
    label= window_situation[0]

    # Store feature and label
    features.append([F1, F2, F3, label, start])

# Convert to DataFrame
KF_features_df= pd.DataFrame(features, columns= [
    'F1_sum_vector_magnitude',
    'F2_orientation_angle',
    'F3_std_magnitude',
    'Situation',
    'window_start_index'
])

KF_features_df.sample(5, random_state= 48)

,F1_sum_vector_magnitude,F2_orientation_angle,F3_std_magnitude,Situation,window_start_index
1151,510.98,1.34,229.42,Fall,172650
3842,493.15,0.47,212.40,Fall,576300
5832,497.97,1.76,7.15,Fall,874800
10870,566.76,0.45,304.52,Not Fall,1630650
5683,502.33,0.34,76.27,Fall,852450
